In [1]:
import time

# Random HUBO generator

In [2]:

import numpy as np

def generate_random_hubo(num_variables, max_order, num_terms, seed=None):
    # Args:
    #    num_variables (int): The total number of binary variables in the HUBO problem.
    #    max_order (int):  The maximum order (degree) of terms in the HUBO problem. 
    #    num_terms (int): The number of terms to generate in the HUBO problem.
    #    seed (int, optional): Random seed for reproducibility. Defaults to None.

    rng = np.random.default_rng(seed)
    hubo = {} # Dict to store the HUBO problem

    for _ in range(num_terms):
        # Randomly choose the order of the term
        order = rng.integers(1, max_order + 1)

        # Randomly select variable indices for the term
        variables = tuple(sorted(rng.choice(num_variables, size=order, replace=False)))

        # Randomly generate a coefficient for the term
        coefficient = rng.uniform(-10, 10)

        # Add the term to the HUBO problem
        hubo[variables] = coefficient

    # Returns:
    #    dict: The generated HUBO problem as a dictionary.
    #          Keys are tuples of variable indices, values are coefficients.

    return hubo


# Example usage
num_variables = 4    # Number of binary variables
max_order = 4        # Maximum order of terms (here cubic terms) --> If set to 2 we get a QUBO problem!
num_terms = 10       # Number of terms in the HUBO
seed = 7            # Random seed for reproducibility of terms and coefficients

rand_hubo_problem = generate_random_hubo(num_variables, max_order, num_terms, seed)

# Print the HUBO problem
print("Generated HUBO Problem:")
for key, value in rand_hubo_problem.items():
    print(f"Term: {key}, Coefficient: {value:.2f}")




Generated HUBO Problem:
Term: (0, 1, 2, 3), Coefficient: -6.80
Term: (1,), Coefficient: -9.89
Term: (0, 2), Coefficient: -1.10
Term: (1, 2), Coefficient: 9.91
Term: (3,), Coefficient: -9.12
Term: (0, 2, 3), Coefficient: 3.84
Term: (2, 3), Coefficient: -0.06


In [3]:
# Print the HUBO problem in its dictionary format
rand_hubo_problem

{(0, 1, 2, 3): -6.795759322843109,
 (1,): -9.894693908688506,
 (0, 2): -1.0984738823470686,
 (1, 2): 9.910005668687852,
 (3,): -9.121159840772332,
 (0, 2, 3): 3.840642417636783,
 (2, 3): -0.06253129212991482}

# Mapping HUBO to QUBO

## Mapping via auxiliary variables == penalty functions (coefficients tuned by hand)

In [4]:

def hubo_to_qubo(hubo):
    #Args:
    #    hubo (dict): The HUBO problem, where keys are tuples of binary variables
    #                 (e.g., (i, j, k)) and values are coefficients, as with the generate_random_hubo
    
    qubo = {} # dict to store the QUBO problem
    aux_var_count = 0  # Counter for auxiliary variables

    for key, coeff in hubo.items():
        if len(key) > 2:  # If the term is higher-order
            # Decompose the higher-order term
            vars_list = list(key)

            # Create an auxiliary variable for the interaction (i.e. penalty function)
            aux_var = num_variables + aux_var_count # first I have named the aux. var. #f"aux_{aux_var_count}", but this is hard to transfrom into qiskit QuadraticProgram object, so now we number them
            aux_var_count += 1

            # Add the new quadratic terms to the QUBO
            for var in vars_list:
                qubo[(var, aux_var)] = coeff / len(vars_list)
            qubo[(aux_var, aux_var)] = -coeff   # Bias term for auxiliary variable
        else:
            # Add quadratic terms directly to the QUBO
            qubo[key] = coeff
        
    #Returns:
    #    dict: The resulting QUBO problem in the form of a dictionary, with the same key/value rules applied.
    #    NOTE: like I have ,entioned before if generate_random_hubo max_order is set to 2, we get QUBO, so this function does nothing!

    return qubo, aux_var_count


# Example: HUBO problem to check if it works (can be done pen and paper)
""" hubo = {
    (0, 1, 2): 3.0,  # A cubic term
    (1, 2): -1.5,    # A quadratic term
    (0,): 2.0        # A linear term
} """

# Convert our random HUBO to QUBO
qubo_from_hubo_pen, aux_var_count = hubo_to_qubo(rand_hubo_problem)
print("QUBO:", qubo_from_hubo_pen)
print("No. of auxiliary variables:", aux_var_count) # see that if we generate a quadratic HUBO (i.e. a QUBO) the prog. will return 0 auxiliary variables




QUBO: {(0, 4): -1.6989398307107773, (1, 4): -1.6989398307107773, (2, 4): -1.6989398307107773, (3, 4): -1.6989398307107773, (4, 4): 6.795759322843109, (1,): -9.894693908688506, (0, 2): -1.0984738823470686, (1, 2): 9.910005668687852, (3,): -9.121159840772332, (0, 5): 1.280214139212261, (2, 5): 1.280214139212261, (3, 5): 1.280214139212261, (5, 5): -3.840642417636783, (2, 3): -0.06253129212991482}
No. of auxiliary variables: 2


## Mapping via penalty functions again, tunable by hand still

In [5]:
import numpy as np

def map_hubo_to_qubo(hubo, num_variables):
    """
    Map a HUBO problem with higher-order terms to a QUBO problem.
    
    Args:
    - hubo (dict): HUBO problem with higher-order terms as key-value pairs.
    - num_variables (int): Number of binary variables in the HUBO problem.
    
    Returns:
    - qubo_matrix (numpy.ndarray): The resulting QUBO matrix.
    """
    # Initialize QUBO matrix
    qubo_matrix = np.zeros((num_variables, num_variables))
    
    aux_var_index = num_variables  # Start index for auxiliary variables
    
    for variables, coeff in hubo.items():
        order = len(variables)
        
        if order == 1:  # Linear terms
            i = variables[0]
            qubo_matrix[i, i] += coeff
            
        elif order == 2:  # Quadratic terms
            i, j = variables
            qubo_matrix[i, j] += coeff
            qubo_matrix[j, i] += coeff  # Symmetric matrix
            
        elif order == 3:  # Cubic terms
            i, j, k = variables
            z_ijk = aux_var_index
            aux_var_index += 1

            # Resize the QUBO matrix to accommodate the auxiliary variable
            if aux_var_index >= qubo_matrix.shape[0]:
                qubo_matrix = np.resize(qubo_matrix, (aux_var_index + 1, aux_var_index + 1))
            
            # Add penalty terms for cubic interaction
            penalty_coeff =  coeff  # Penalty coefficient
            qubo_matrix = add_penalty_terms(qubo_matrix, z_ijk, [i, j, k], penalty_coeff, cubic=True)
            
            # Update QUBO matrix for cubic term
            qubo_matrix[i, j] += coeff * z_ijk
            qubo_matrix[j, k] += coeff * z_ijk
            qubo_matrix[i, k] += coeff * z_ijk
            
        # Additional cases can be added for quartic terms or higher-order terms.
    
    return qubo_matrix

def add_penalty_terms(Q, aux_var, vars, penalty_coeff, cubic=True):
    """
    Add penalty terms to the QUBO matrix for enforcing higher-order terms (cubic or quartic).
    
    Args:
    - Q: The QUBO matrix.
    - aux_var: The index of the auxiliary variable.
    - vars: The indices of the variables involved in the interaction.
    - penalty_coeff: The penalty coefficient.
    - cubic: Whether the penalty is for cubic (True) or quartic (False).
    
    Returns:
    - Updated QUBO matrix.
    """
    if cubic:
        # For cubic terms, introduce quadratic penalties
        Q[aux_var, aux_var] = penalty_coeff
        for v in vars:
            Q[aux_var, v] = -penalty_coeff
            Q[v, aux_var] = -penalty_coeff
    else:
        # For quartic terms, introduce quadratic penalties
        Q[aux_var, aux_var] = penalty_coeff
        for v in vars:
            Q[aux_var, v] = -penalty_coeff
            Q[v, aux_var] = -penalty_coeff

    return Q

def matr_to_dict(input_matrix):
    upper_triangle_dict = {(i, j): qubo_matrix[i][j] 
                            for i in range(len(qubo_matrix)) 
                            for j in range(i, len(qubo_matrix[i]))}  # Only for j >= i as we should have a symmetrical matrix
    return upper_triangle_dict

# Number of binary variables
num_variables = 3

# Convert HUBO problem to QUBO matrix
qubo_matrix = map_hubo_to_qubo(rand_hubo_problem, num_variables)
qubo_dict = matr_to_dict(qubo_matrix)

# Print the resulting QUBO Matrix
print("Converted QUBO Matrix:")
print(qubo_matrix)
# Print the resulting QUBO dictionary
print("Converted QUBO dictionary:")
print(qubo_dict)


IndexError: index 3 is out of bounds for axis 0 with size 3

## Mapping via the dimod (dwave) algorithm

Automatic conversion, Auomatically handled auxiliary variables

In [6]:
import dimod

poly = dimod.BinaryPolynomial.from_hubo(rand_hubo_problem) # Construct a binary polynomial from a higher-order unconstrained binary optimization (HUBO) problem.

In [7]:
quadratic_from_hubo = dimod.make_quadratic(rand_hubo_problem, 10.0, dimod.BINARY) # Create a binary quadratic model from a higher order polynomial.

## Mapping via Qubovert package

Automatic conversion, Auomatically handled auxiliary variables

In [8]:
from qubovert import PUBO

# Define a HUBO
hubo = PUBO(rand_hubo_problem)  # Example: x0 * x1 * x2 + 2 * x1 * x2 * x3

# Convert HUBO to QUBO
qubo_qubovert = hubo.to_qubo()

# Print results
print("QUBO:", qubo_qubovert)

qubo_variables = set(var for term in qubo_qubovert for var in term)


QUBO: {(4,): 37.90920522143968, (0, 2): 11.537927858132823, (0, 4): -25.272803480959784, (2, 4): -25.272803480959784, (5,): 23.387277968529325, (1, 3): 7.795759322843109, (1, 5): -15.591518645686218, (3, 5): -15.591518645686218, (4, 5): -6.795759322843109, (1,): -9.894693908688506, (1, 2): 9.910005668687852, (3,): -9.121159840772332, (3, 4): 3.840642417636783, (2, 3): -0.06253129212991482}


## Mapping via tnreason polynomialcore engine

In [9]:

import docplex
from qiskit_optimization.translators import from_docplex_mp

from docplex.mp.model import Model
from tnreason import engine

def poly_to_cplex_model(polyCore):
    
    #Convert a HUBO into an ILP by introducing inequality constraints.
    #All terms are replaced by slack variables and assosiated constraints.

    #Args:
    #    polyCore (PolynomialCore)
    #Returns:
    #    model: A docplex model. Can be then fed into QUBO solvers

    model = Model("ILP_to_QUBO")

    variableDict = {
        color: model.binary_var(name=color) for color in polyCore.colors
    }

    slackVariableDict = {
        "slack" + str(j): model.binary_var(name="slack" + str(j)) for j in range(len(polyCore.values))
    }

    for j, entry in enumerate(polyCore.values):
        lowBound = 1
        for var in entry[1]:
            if entry[1][var] == 1:
                lowBound = lowBound + variableDict[var] - 1
                model.add_constraint(slackVariableDict["slack" + str(j)] <= variableDict[var])
            elif entry[1][var] == 0:
                lowBound = lowBound - variableDict[var]
                model.add_constraint(slackVariableDict["slack" + str(j)] <= (1 - variableDict[var]))
            else:
                raise ValueError("Index {} not supported, binary only!".format(entry[1][var]))
        model.add_constraint(lowBound <= slackVariableDict["slack" + str(j)])

    objective = 0
    for j, entry in enumerate(polyCore.values):
        objective = objective + entry[0] * slackVariableDict["slack" + str(j)]

    model.maximize(objective)
    return model

colors = ["a","b","c","d"]

polyCore = engine.get_core("PolynomialCore")(values=[(2, {"a":1, "b":1, "c":1}), (-1, {"a":1,"b":1,"d":1})], colors=colors, shape=[2 for key in colors])
#polyCore = engine.get_core("PolynomialCore")(values=[(2, {"a":1, "b":1}), (-1, {"a":1,"d":1}), (1, {"c":1,"d":1})], colors=colors, shape=[2 for key in colors])
#polyCore = engine.get_core("PolynomialCore")(values=[(1, {"a":1}), (1, {"b":1}), (1, {"d":1}), (1, {"c":1})], colors=colors, shape=[2 for key in colors])
#polyCore = engine.get_core("PolynomialCore")(values=[(1, {"a":1})], colors=["a"], shape=[2 for key in ["a"]])
#polyCore = engine.get_core("PolynomialCore")(values=[(1, {"a":1, "b":1})], colors=["a","b"], shape=[2 for key in ["a","b"]])
#polyCore = engine.get_core("PolynomialCore")(values=[(1, {"a":1})], colors=["a"], shape=[2 for key in ["a"]])
model = poly_to_cplex_model(polyCore)
model.prettyprint()

// This file has been generated by DOcplex
// model name is: ILP_to_QUBO
// single vars section
dvar bool a;
dvar bool b;
dvar bool c;
dvar bool d;
dvar bool slack0;
dvar bool slack1;

maximize
 2 slack0 - slack1;
 
subject to {
 slack0 <= a;
 slack0 <= b;
 slack0 <= c;
 a + b + c -2 <= slack0;
 slack1 <= a;
 slack1 <= b;
 slack1 <= d;
 a + b + d -2 <= slack1;

}


# Try Creating and solving QUBOs with qiskit

For this to work we need to slice up the qubo matrix and feed the linear / quadratic parts separately to the QuadraticProgram()

In [10]:
def slicer(input_dict): # slice our dict of input on the size of keys (i.e. the terms being Quadratic / Linear)

    quadratic_terms = {} # make two dictionaries for quadratic and linear terms
    linear_terms = {}

    for i in range(len(input_dict)):
        if len(list(input_dict.keys())[i]) == 2:
            quadratic_terms[list(input_dict.keys())[i]] = list(input_dict.values())[i]

    for i in range(len(input_dict)):
        if len(list(input_dict.keys())[i]) == 1:
            linear_terms[list(input_dict.keys())[i]] = list(input_dict.values())[i]
                                                                  
    return quadratic_terms, linear_terms

In [11]:
q, l = slicer(qubo_qubovert)

In [12]:
l

{(4,): 37.90920522143968,
 (5,): 23.387277968529325,
 (1,): -9.894693908688506,
 (3,): -9.121159840772332}

In [13]:
q

{(0, 2): 11.537927858132823,
 (0, 4): -25.272803480959784,
 (2, 4): -25.272803480959784,
 (1, 3): 7.795759322843109,
 (1, 5): -15.591518645686218,
 (3, 5): -15.591518645686218,
 (4, 5): -6.795759322843109,
 (1, 2): 9.910005668687852,
 (3, 4): 3.840642417636783,
 (2, 3): -0.06253129212991482}

In [14]:
from qiskit_aer import Aer
from qiskit_algorithms import QAOA, VQE
from qiskit_algorithms.optimizers import COBYLA, SPSA, ADAM, NELDER_MEAD, L_BFGS_B
from qiskit.circuit.library import TwoLocal
from qiskit.primitives import Sampler
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.algorithms import MinimumEigenOptimizer

# Step 1: Define the QUBO problem
qubo = QuadraticProgram()
# Add binary variables from our previously made random HUBO turned QUBO
for i in range(0, len(qubo_variables)):
    qubo.binary_var(f'x{i}')
""" for i in range(0, aux_var_count):
    qubo.binary_var(f'aux{i}') """


# Define the QUBO objective (minimize/maximuze usually)
qubo.minimize(linear=l, quadratic=q)  

In [15]:
qubo.prettyprint() #see if it has been fed correctly

'Problem name: \n\nMinimize\n  11.537927858132823*x0*x2 - 25.272803480959784*x0*x4 + 9.910005668687852*x1*x2\n  + 7.795759322843109*x1*x3 - 15.591518645686218*x1*x5\n  - 0.06253129212991482*x2*x3 - 25.272803480959784*x2*x4\n  + 3.840642417636783*x3*x4 - 15.591518645686218*x3*x5 - 6.795759322843109*x4*x5\n  - 9.894693908688506*x1 - 9.121159840772332*x3 + 37.90920522143968*x4\n  + 23.387277968529325*x5\n\nSubject to\n  No constraints\n\n  Binary variables (6)\n    x0 x1 x2 x3 x4 x5\n'

In [16]:
qubo_from_hubo_pen # our randomly generated HUBO turned QUBO with the added penalty terms

{(0, 4): -1.6989398307107773,
 (1, 4): -1.6989398307107773,
 (2, 4): -1.6989398307107773,
 (3, 4): -1.6989398307107773,
 (4, 4): 6.795759322843109,
 (1,): -9.894693908688506,
 (0, 2): -1.0984738823470686,
 (1, 2): 9.910005668687852,
 (3,): -9.121159840772332,
 (0, 5): 1.280214139212261,
 (2, 5): 1.280214139212261,
 (3, 5): 1.280214139212261,
 (5, 5): -3.840642417636783,
 (2, 3): -0.06253129212991482}

In [17]:
qubo_qubovert # our randomly generated HUBO turned QUBO via the qubovert package

{(4,): 37.90920522143968,
 (0, 2): 11.537927858132823,
 (0, 4): -25.272803480959784,
 (2, 4): -25.272803480959784,
 (5,): 23.387277968529325,
 (1, 3): 7.795759322843109,
 (1, 5): -15.591518645686218,
 (3, 5): -15.591518645686218,
 (4, 5): -6.795759322843109,
 (1,): -9.894693908688506,
 (1, 2): 9.910005668687852,
 (3,): -9.121159840772332,
 (3, 4): 3.840642417636783,
 (2, 3): -0.06253129212991482}

In [18]:
quadratic_from_hubo # our randomly generated HUBO turned QUBO via the d-wave package

BinaryQuadraticModel({0: 0.0, 2: 0.0, '0*2': 30.0, 1: -9.894693908688506, '0*2*1': 30.0, 3: -9.121159840772332}, {(2, 0): 8.901526117652931, ('0*2', 0): -20.0, ('0*2', 2): -20.0, (1, 2): 9.910005668687852, (1, '0*2'): 10.0, ('0*2*1', '0*2'): -20.0, ('0*2*1', 1): -20.0, (3, 2): -0.06253129212991482, (3, '0*2'): 3.840642417636783, (3, '0*2*1'): -6.795759322843109}, 0.0, 'BINARY')

## Solving via Qiskit classical methods (Gurobi)

In [19]:
from qiskit_optimization.algorithms import GurobiOptimizer

start_classical = time.time()
# Use the GurobiOptimizer to find the solution
exact_solver = GurobiOptimizer()
result = exact_solver.solve(qubo)

end_classical = time.time()
# Display the result

print("Optimal solution(s):")
solutions = [s for s in result.samples if s.fval == result.fval]
for solution in solutions:
    print(solution)
    
print("Optimal value:", result.fval)
print("Calculated in:", end_classical - start_classical, "seconds.")

Restricted license - for non-production use only - expires 2026-11-23


Optimal solution(s):
SolutionSample(x=[0.0, 1.0, 0.0, 1.0, 0.0, 1.0], fval=-19.015853749460838, probability=1.0, status=<OptimizationResultStatus.SUCCESS: 0>)
Optimal value: -19.015853749460838
Calculated in: 0.04634499549865723 seconds.


## Solving via Qiskit quantum methods (QAOA)

In [20]:
# Step 2.B: Set up the QAOA solver
sampler = Sampler()  # Sampling primitive for Qiskit
qaoa = QAOA(sampler=sampler, optimizer = COBYLA(), reps=10)  # reps = depth of QAOA

start_quantum = time.time()
# Use the QAOA algorithm to solve the problem
optimizer = MinimumEigenOptimizer(qaoa)
result = optimizer.solve(qubo)

end_quantum = time.time()
# Display the result
print("Optimal solution(s):")
solutions = [s for s in result.samples if s.fval == result.fval]
for solution in solutions:
    print(solution)

print("Optimal value:", result.fval)
print("Calculated in:", end_quantum - start_quantum, "seconds.")

C:\Users\Botond\AppData\Local\Temp\ipykernel_2300\3582856058.py:2: DeprecationWarning: The class ``qiskit.primitives.sampler.Sampler`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseSamplerV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Sampler` class is `StatevectorSampler`.
  sampler = Sampler()  # Sampling primitive for Qiskit


Optimal solution(s):
SolutionSample(x=array([0., 1., 0., 1., 0., 1.]), fval=-19.015853749460838, probability=0.0434276555263436, status=<OptimizationResultStatus.SUCCESS: 0>)
SolutionSample(x=array([1., 1., 0., 1., 0., 1.]), fval=-19.015853749460838, probability=0.1641041132163319, status=<OptimizationResultStatus.SUCCESS: 0>)
Optimal value: -19.015853749460838
Calculated in: 33.724891662597656 seconds.


## Solve HUBO directly using Using Dimod (D-Wave's Python SDK)

In [21]:
import dimod
from dimod import ExactSolver
solver = ExactSolver() # brute force approach

# Solve the HUBO problem using the solver
solution = solver.sample(poly)

sample_list = []
low_en = float()
low_en_sample = _
# Print the solution and its energy
for sample, energy in solution.data(['sample', 'energy']):
    sample_list.append((sample, energy))

for i in range (0, len(sample_list)):
    if sample_list[i][1] < low_en:
        low_en = sample_list[i][1]
        low_en_sample = sample_list[i][0]

print("Sample:", low_en_sample, "Lowest Energy:", low_en)



Sample: {0: 0, 1: 1, 2: 0, 3: 1} Lowest Energy: -19.015853749460838


## Solve QUBO via simulated annealing (D-Wave's Python SDK)

In [22]:
from dwave.samplers import SimulatedAnnealingSampler
from dimod import BinaryQuadraticModel
#bqm = BinaryQuadraticModel.from_qubo(qubo_from_hubo_matrix)

sampler = SimulatedAnnealingSampler()
response = sampler.sample(quadratic_from_hubo, num_reads=100)

# Print the best solution
best_solution = response.first.sample
best_energy = response.first.energy

print("Best solution:", best_solution)
print("Best energy:", best_energy)

Best solution: {0: 0, 2: 0, '0*2': 0, 1: 1, '0*2*1': 0, 3: 1}
Best energy: -19.015853749460838


In [23]:
# Print all solutions
for sample, energy, occurrences in response.data(fields=['sample', 'energy', 'num_occurrences']):
    print("Solution:", sample, "Energy:", energy, "Occurrences:", occurrences)


Solution: {0: 0, 2: 0, '0*2': 0, 1: 1, '0*2*1': 0, 3: 1} Energy: -19.015853749460838 Occurrences: 1
Solution: {0: 0, 2: 0, '0*2': 0, 1: 1, '0*2*1': 0, 3: 1} Energy: -19.015853749460838 Occurrences: 1
Solution: {0: 1, 2: 0, '0*2': 0, 1: 1, '0*2*1': 0, 3: 1} Energy: -19.015853749460838 Occurrences: 1
Solution: {0: 1, 2: 0, '0*2': 0, 1: 1, '0*2*1': 0, 3: 1} Energy: -19.015853749460838 Occurrences: 1
Solution: {0: 1, 2: 0, '0*2': 0, 1: 1, '0*2*1': 0, 3: 1} Energy: -19.015853749460838 Occurrences: 1
Solution: {0: 0, 2: 0, '0*2': 0, 1: 1, '0*2*1': 0, 3: 1} Energy: -19.015853749460838 Occurrences: 1
Solution: {0: 0, 2: 0, '0*2': 0, 1: 1, '0*2*1': 0, 3: 1} Energy: -19.015853749460838 Occurrences: 1
Solution: {0: 0, 2: 0, '0*2': 0, 1: 1, '0*2*1': 0, 3: 1} Energy: -19.015853749460838 Occurrences: 1
Solution: {0: 1, 2: 0, '0*2': 0, 1: 1, '0*2*1': 0, 3: 1} Energy: -19.015853749460838 Occurrences: 1
Solution: {0: 1, 2: 0, '0*2': 0, 1: 1, '0*2*1': 0, 3: 1} Energy: -19.015853749460838 Occurrences: 1


## Exact solver (Brute force)

In [24]:
import itertools

def evaluate_qubo(qubo, assignment):
    
    #Evaluate the QUBO objective function for a given binary assignment.

    #Args:
    #    qubo (dict): The QUBO problem as a dictionary.
    #    assignment (dict): A dictionary mapping variable names to binary values (0 or 1).

    #Returns:
    #    float: The value of the objective function.

    # Evaluate the objective function by multiplying the coefficients with the assigned values and summing them up.

    # Note: The QUBO is in the form of a dictionary where keys are tuples of variable names (e.g., ('x0', 'x1')) and values are coefficients.
    # This function assumes that the QUBO is symmetric, i.e., qubo[(v1, v2)] == qubo[(v2, v1)].

    # Example:
    objective_value = 0
    for (vars_, coeff) in qubo.items():
        product = coeff
        for var in vars_:
            product *= assignment[var]
        objective_value += product
    return objective_value

start_brute = time.time()
def brute_force_solve_qubo(qubo):
    
    #Solve the QUBO problem using a brute-force approach.

    #Args:
    #    qubo (dict): The QUBO problem as a dictionary.

    #Returns:
    #    dict: The optimal binary assignment.
    #    float: The minimum value of the objective function.

    # Extract all unique variables from the QUBO dictionary keys.
    variables = set()
    for key in qubo.keys():
        variables.update(key)

    # Generate all possible binary assignments
    num_variables = len(variables)
    variable_list = list(variables)
    all_assignments = itertools.product([0, 1], repeat=num_variables)

    # Evaluate all assignments and find the minimum
    min_value = float('inf')    # value of the minimum
    optimal_assignment = []   # the optimal binary assignment of variables

    for assignment_tuple in all_assignments:
        assignment = dict(zip(variable_list, assignment_tuple))
        value = evaluate_qubo(qubo, assignment)
        if value < min_value:
            min_value = value
            optimal_assignment.append(assignment)

    return optimal_assignment, min_value

# Solve the QUBO problem
optimal_assignment, min_value = brute_force_solve_qubo(rand_hubo_problem)

end_brute = time.time()
# Print the results
print("Optimal Assignment(s):")
for el in optimal_assignment:
    print(el)

print("Minimum Objective Value:", min_value)
print("Calculated in:", end_brute - start_brute, "seconds.")


Optimal Assignment(s):
{0: 0, 1: 0, 2: 0, 3: 0}
{0: 0, 1: 0, 2: 0, 3: 1}
{0: 0, 1: 0, 2: 1, 3: 1}
{0: 0, 1: 1, 2: 0, 3: 0}
{0: 0, 1: 1, 2: 0, 3: 1}
Minimum Objective Value: -19.015853749460838
Calculated in: 0.0 seconds.


# Try Creating and solving QUBOs with D-wave

In [16]:
import dwave.system
dwave.system.Client.from_config(token='DEV-19676b6aea481ee3dcec55055b81db900d5c6544')

AttributeError: module 'dwave.system' has no attribute 'Client'

In [40]:
from qiskit_optimization import QuadraticProgram
from dimod import BinaryQuadraticModel

def qiskit_to_dwave_bqm(qp: QuadraticProgram) -> BinaryQuadraticModel:
    """
    Convert a Qiskit QuadraticProgram to a D-Wave BinaryQuadraticModel.
    
    Args:
        qp (QuadraticProgram): The Qiskit QuadraticProgram to convert.

    Returns:
        BinaryQuadraticModel: The corresponding BinaryQuadraticModel.
    """
    
    # Initialize linear and quadratic coefficients
    linear = {}
    quadratic = {}
    offset = qp.objective.constant

    # Extract linear coefficients
    for i, var in enumerate(qp.variables):
        linear[var.name] = qp.objective.linear.get(var.name, 0.0)

    # Extract quadratic coefficients
    for (i, j), coeff in qp.objective.quadratic.items():
        quadratic[(i, j)] = coeff

    # Create BinaryQuadraticModel
    bqm = BinaryQuadraticModel(linear, quadratic, offset, vartype='BINARY')
    return bqm

In [41]:
bqm = qiskit_to_dwave_bqm(qubo)

AttributeError: 'LinearExpression' object has no attribute 'get'

In [90]:
def qiskit_to_dwave(linear_terms): # slice our dict of input on the size of keys (i.e. the terms being Quadratic / Linear)

    linear_terms_dwave = {} # make two dictionaries for quadratic and linear terms

    for i in range(len(linear_terms)):
        linear_terms_dwave[(list(linear_terms.keys())[i], list(linear_terms.keys())[i])] = list(linear_terms.values())[i]

                                                                  
    return linear_terms

l_d = qiskit_to_dwave(l)

In [91]:
list(l.keys())[0]

(1,)

In [92]:
l_d

{(1,): 1.0699470414898489}

## Classical solution via D-wave interface

In [93]:
qubo_from_hubo_pen

{(0, 3): -0.3661579607823562,
 (1, 3): -0.3661579607823562,
 (2, 3): -0.3661579607823562,
 (3, 3): 1.0984738823470686,
 (0, 1): -0.641300943125584,
 (1,): 1.0699470414898489}

In [94]:
from dwave.samplers import SimulatedAnnealingSampler
from dimod import BinaryQuadraticModel

Q = {(0, 0): 1, (1, 1): 2, (0, 1): -4}  # QUBO coefficients
bqm = BinaryQuadraticModel.from_qubo(qubo_from_hubo_pen)

sampler = SimulatedAnnealingSampler()
response = sampler.sample(bqm)

print(response.first.sample)  # Optimal solution
print(response.first.energy)

ValueError: not enough values to unpack (expected 2, got 1)

In [95]:
import numpy as np
import dimod

# Define the QUBO matrix
# Example QUBO matrix for the problem: minimize x0^2 + x0*x1 - 2*x1^2
Q = {
    ('x0', 'x0'): 1,    # x0^2 term
    ('x0', 'x1'): 1,    # x0*x1 term
    ('x1', 'x1'): -2    # x1^2 term
}

# Use dimod to define a QUBO model
qubo = dimod.BinaryQuadraticModel.from_qubo(qubo_from_hubo_pen)

# Solve the QUBO classically using a brute force solver
class_sampler = dimod.ExactSolver()
class_solution = class_sampler.sample(qubo)

# Extract the results
print("Solutions:")
for sample, energy in solution.data(['sample', 'energy']):
    print(f"Sample: {class_sampler}, Energy: {energy}")

# Find the optimal solution
optimal_class_sampler = solution.first.sample
optimal_class_energy = solution.first.energy
print(f"\nOptimal Solution: {optimal_class_sampler}, Energy: {optimal_energy}")
optimal_class_energy

ValueError: not enough values to unpack (expected 2, got 1)

## Quantum solution via D-wave interface

In [33]:
from dwave.system import DWaveSampler, EmbeddingComposite

quantum_sampler = DWaveSampler(solver={'qpu': True})  # Use a real QPU solver
composite_quatum_sampler = EmbeddingComposite(quantum_sampler)  # Embedding to QPU

sampler = EmbeddingComposite(DWaveSampler(token='DEV-19676b6aea481ee3dcec55055b81db900d5c6544', solver={'qpu': True}))

# Sample the QUBO using the D-Wave solver
response = sampler.sample(bqm, num_reads=100)

# Print the solution with lowest energy
print("Solution with lowest energy:", response.first.sample)
print("Energy of the solution:", response.first.energy)

ValueError: API token not defined

In [96]:
import numpy as np

def generate_qubo_from_hubo(hubo, num_variables):
    """
    Convert a HUBO matrix (stored as a dictionary of terms) into a QUBO matrix.

    Args:
    - hubo (dict): The HUBO problem, with variable indices as keys and coefficients as values.
    - num_variables (int): The number of binary variables.

    Returns:
    - Q (numpy.ndarray): The resulting QUBO matrix.
    """
    # Initialize QUBO matrix
    Q = np.zeros((num_variables, num_variables))  
    aux_var_index = num_variables  # Start index for auxiliary variables

    for variables, coeff in hubo.items():
        order = len(variables)

        if order == 2:  # Quadratic term
            i, j = variables
            Q[i, j] += coeff
            Q[j, i] += coeff  # Symmetric matrix

        elif order == 3:  # Cubic term
            i, j, k = variables
            z_ijk = aux_var_index
            aux_var_index += 1

            # Resize the QUBO matrix to accommodate the auxiliary variable
            if aux_var_index >= Q.shape[0]:
                Q = np.resize(Q, (aux_var_index + 1, aux_var_index + 1))

            # Add penalty terms for cubic interaction
            penalty_coeff = 1e3  # Penalty coefficient
            Q = add_penalty_terms(Q, z_ijk, [i, j, k], penalty_coeff, cubic=True)
            Q[i, j] += coeff * z_ijk
            Q[j, k] += coeff * z_ijk
            Q[i, k] += coeff * z_ijk

        elif order == 4:  # Quartic term
            i, j, k, l = variables
            w_ijkl = aux_var_index
            aux_var_index += 1

            # Resize the QUBO matrix to accommodate the auxiliary variable
            if aux_var_index >= Q.shape[0]:
                Q = np.resize(Q, (aux_var_index + 1, aux_var_index + 1))

            # Add penalty terms for quartic interaction
            penalty_coeff = 1e3  # Penalty coefficient
            Q = add_penalty_terms(Q, w_ijkl, [i, j, k, l], penalty_coeff, cubic=False)
            Q[i, j] += coeff * w_ijkl
            Q[j, k] += coeff * w_ijkl
            Q[k, l] += coeff * w_ijkl
            Q[i, l] += coeff * w_ijkl

    return Q

def add_penalty_terms(Q, aux_var, vars, penalty_coeff, cubic=True):
    """
    Add penalty terms to the QUBO matrix for enforcing higher-order terms (cubic or quartic).

    Args:
    - Q: The QUBO matrix.
    - aux_var: The index of the auxiliary variable.
    - vars: The indices of the variables involved in the interaction.
    - penalty_coeff: The penalty coefficient.
    - cubic: Whether the penalty is for cubic (True) or quartic (False).

    Returns:
    - Updated QUBO matrix.
    """
    if cubic:
        # For cubic terms, introduce quadratic penalties
        Q[aux_var, aux_var] = penalty_coeff
        for v in vars:
            Q[aux_var, v] = -penalty_coeff
            Q[v, aux_var] = -penalty_coeff
    else:
        # For quartic terms, introduce quadratic penalties
        Q[aux_var, aux_var] = penalty_coeff
        for v in vars:
            Q[aux_var, v] = -penalty_coeff
            Q[v, aux_var] = -penalty_coeff

    return Q



# Convert HUBO problem to QUBO matrix
qubo_matrix = generate_qubo_from_hubo(rand_hubo_problem, num_variables)

# Print the QUBO matrix
print("Converted QUBO Matrix:")
print(qubo_matrix)


Converted QUBO Matrix:
[[ 0.00000000e+00 -3.93672259e+00 -3.29542165e+00 -1.00000000e+03
   0.00000000e+00]
 [-6.41300943e-01  0.00000000e+00 -3.29542165e+00 -1.00000000e+03
   0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.00000000e+03
   0.00000000e+00]
 [-1.00000000e+03 -1.00000000e+03 -1.00000000e+03  1.00000000e+03
   0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]]


In [97]:
rand_hubo_problem

{(0, 1, 2): -1.0984738823470686,
 (0, 1): -0.641300943125584,
 (1,): 1.0699470414898489}

In [118]:
import numpy as np

def map_hubo_to_qubo(hubo, num_variables):
    """
    Map a HUBO problem with higher-order terms to a QUBO problem.
    
    Args:
    - hubo (dict): HUBO problem with higher-order terms as key-value pairs.
    - num_variables (int): Number of binary variables in the HUBO problem.
    
    Returns:
    - qubo_matrix (numpy.ndarray): The resulting QUBO matrix.
    """
    # Initialize QUBO matrix
    qubo_matrix = np.zeros((num_variables, num_variables))
    
    aux_var_index = num_variables  # Start index for auxiliary variables
    
    for variables, coeff in hubo.items():
        order = len(variables)
        
        if order == 1:  # Linear terms
            i = variables[0]
            qubo_matrix[i, i] += coeff
            
        elif order == 2:  # Quadratic terms
            i, j = variables
            qubo_matrix[i, j] += coeff
            qubo_matrix[j, i] += coeff  # Symmetric matrix
            
        elif order == 3:  # Cubic terms
            i, j, k = variables
            z_ijk = aux_var_index
            aux_var_index += 1

            # Resize the QUBO matrix to accommodate the auxiliary variable
            if aux_var_index >= qubo_matrix.shape[0]:
                qubo_matrix = np.resize(qubo_matrix, (aux_var_index + 1, aux_var_index + 1))
            
            # Add penalty terms for cubic interaction
            penalty_coeff = 1e3  # Penalty coefficient
            qubo_matrix = add_penalty_terms(qubo_matrix, z_ijk, [i, j, k], penalty_coeff, cubic=True)
            
            # Update QUBO matrix for cubic term
            qubo_matrix[i, j] += coeff * z_ijk
            qubo_matrix[j, k] += coeff * z_ijk
            qubo_matrix[i, k] += coeff * z_ijk
            
        # Additional cases can be added for quartic terms or higher-order terms.
    
    return qubo_matrix

def add_penalty_terms(Q, aux_var, vars, penalty_coeff, cubic=True):
    """
    Add penalty terms to the QUBO matrix for enforcing higher-order terms (cubic or quartic).
    
    Args:
    - Q: The QUBO matrix.
    - aux_var: The index of the auxiliary variable.
    - vars: The indices of the variables involved in the interaction.
    - penalty_coeff: The penalty coefficient.
    - cubic: Whether the penalty is for cubic (True) or quartic (False).
    
    Returns:
    - Updated QUBO matrix.
    """
    if cubic:
        # For cubic terms, introduce quadratic penalties
        Q[aux_var, aux_var] = penalty_coeff
        for v in vars:
            Q[aux_var, v] = -penalty_coeff
            Q[v, aux_var] = -penalty_coeff
    else:
        # For quartic terms, introduce quadratic penalties
        Q[aux_var, aux_var] = penalty_coeff
        for v in vars:
            Q[aux_var, v] = -penalty_coeff
            Q[v, aux_var] = -penalty_coeff

    return Q

# Number of binary variables
num_variables = 3

# Convert HUBO problem to QUBO matrix
qubo_matrix = map_hubo_to_qubo(rand_hubo_problem, num_variables)

# Print the resulting QUBO matrix
print("Converted QUBO Matrix:")
print(qubo_matrix)


Converted QUBO Matrix:
[[ 0.00000000e+00 -3.93672259e+00 -3.29542165e+00 -1.00000000e+03
   0.00000000e+00]
 [-6.41300943e-01  1.06994704e+00 -3.29542165e+00 -1.00000000e+03
   0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.00000000e+03
   0.00000000e+00]
 [-1.00000000e+03 -1.00000000e+03 -1.00000000e+03  1.00000000e+03
   0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]]


In [99]:
upper_triangle_dict = {(i, j): qubo_matrix[i][j] 
                       for i in range(len(qubo_matrix)) 
                       for j in range(i, len(qubo_matrix[i]))}  # Only for j >= i

print(upper_triangle_dict)


{(0, 0): 0.0, (0, 1): -3.9367225901667897, (0, 2): -3.2954216470412057, (0, 3): -1000.0, (0, 4): 0.0, (1, 1): 1.0699470414898489, (1, 2): -3.2954216470412057, (1, 3): -1000.0, (1, 4): 0.0, (2, 2): 0.0, (2, 3): -1000.0, (2, 4): 0.0, (3, 3): 1000.0, (3, 4): 0.0, (4, 4): 0.0}


In [130]:
for i in range(len(qubo_matrix)):
    for j in range(i, len(qubo_matrix[i])):
        if i == j:
            upper_triangle_dict.update({(i): qubo_matrix[i][j]})  
        else: 
            upper_triangle_dict.update({(i, j): qubo_matrix[i][j]})  # Only for j >= i

print(upper_triangle_dict)

{(0, 0): 0.0, (0, 1): -3.9367225901667897, (0, 2): -3.2954216470412057, (0, 3): -1000.0, (0, 4): 0.0, (1, 1): 1.0699470414898489, (1, 2): -3.2954216470412057, (1, 3): -1000.0, (1, 4): 0.0, (2, 2): 0.0, (2, 3): -1000.0, (2, 4): 0.0, (3, 3): 1000.0, (3, 4): 0.0, (4, 4): 0.0, (0, <class 'numpy.ndarray'>): 0.0, (1, <class 'numpy.ndarray'>): 1.0699470414898489, (2, <class 'numpy.ndarray'>): 0.0, (3, <class 'numpy.ndarray'>): 1000.0, (4, <class 'numpy.ndarray'>): 0.0, (0,): 0.0, (1,): 1.0699470414898489, (2,): 0.0, (3,): 1000.0, (4,): 0.0, 0: 0.0, 1: 1.0699470414898489, 2: 0.0, 3: 1000.0, 4: 0.0}


In [134]:
list(upper_triangle_dict.keys())[0][0] == list(upper_triangle_dict.keys())[0][1]

True

In [102]:
import numpy as np
import dimod

# Define the QUBO matrix
# Example QUBO matrix for the problem: minimize x0^2 + x0*x1 - 2*x1^2
Q = {
    ('x0', 'x0'): 1,    # x0^2 term
    ('x0', 'x1'): 1,    # x0*x1 term
    ('x1', 'x1'): -2    # x1^2 term
}

# Use dimod to define a QUBO model
qubo = dimod.BinaryQuadraticModel.from_qubo(upper_triangle_dict)

# Solve the QUBO classically using a brute force solver
class_sampler = dimod.ExactSolver()
class_solution = class_sampler.sample(qubo)

# Extract the results
print("Solutions:")
for sample, energy in solution.data(['sample', 'energy']):
    print(f"Sample: {class_sampler}, Energy: {energy}")

# Find the optimal solution
optimal_class_sampler = solution.first.sample
optimal_class_energy = solution.first.energy



Solutions:
Sample: <dimod.reference.samplers.exact_solver.ExactSolver object at 0x00000294CBB57970>, Energy: -0.6698277839828037
Sample: <dimod.reference.samplers.exact_solver.ExactSolver object at 0x00000294CBB57970>, Energy: 0.0
Sample: <dimod.reference.samplers.exact_solver.ExactSolver object at 0x00000294CBB57970>, Energy: 0.0
Sample: <dimod.reference.samplers.exact_solver.ExactSolver object at 0x00000294CBB57970>, Energy: 0.0
Sample: <dimod.reference.samplers.exact_solver.ExactSolver object at 0x00000294CBB57970>, Energy: 0.0
Sample: <dimod.reference.samplers.exact_solver.ExactSolver object at 0x00000294CBB57970>, Energy: 0.4286460983642648
Sample: <dimod.reference.samplers.exact_solver.ExactSolver object at 0x00000294CBB57970>, Energy: 1.0699470414898489
Sample: <dimod.reference.samplers.exact_solver.ExactSolver object at 0x00000294CBB57970>, Energy: 1.0699470414898489


In [54]:
qubo_from_hubo_pen

{(0, 3): -0.3661579607823562,
 (1, 3): -0.3661579607823562,
 (2, 3): -0.3661579607823562,
 (3, 3): 1.0984738823470686,
 (0, 1): -0.641300943125584,
 (1,): 1.0699470414898489}

In [120]:
type(qubo_matrix)

numpy.ndarray

l

In [110]:
import numpy as np
from qiskit_optimization import QuadraticProgram

def convert_hubo_to_qubo(hubo, num_variables):
    # Create a QUBO (QuadraticProgram) object
    qp = QuadraticProgram()

    # Add binary variables to the QUBO problem
    for i in range(num_variables):
        qp.binary_var(name=f'x{i}')

    # Create linear and quadratic terms
    linear_terms = {}
    quadratic_terms = {}

    qp_lin = {}
    qp_quad = {}

    for term, coeff in hubo.items():
        if len(term) == 1:  # Linear term (single variable)
            linear_terms[term[0]] = linear_terms.get(term[0], 0) + coeff
        elif len(term) == 2:  # Quadratic term (pair of variables)
            i, j = term
            quadratic_terms[(i, j)] = quadratic_terms.get((i, j), 0) + coeff
        else:
            # Skip higher-order terms (degree > 2)
            continue

    # Add linear terms to the QUBO
    for i, coeff in linear_terms.items():
        qp.minimize(qp_lin.update({(i,) : coeff}), qp_quad)

    # Add quadratic terms to the QUBO
    for (i, j), coeff in quadratic_terms.items():
        qp.minimize(qp_lin, qp_quad.update({(i,j) : coeff}))

    return qp




# Print the generated HUBO problem
print("Generated HUBO Problem:")
for key, value in rand_hubo_problem.items():
    print(f"Term: {key}, Coefficient: {value:.2f}")

# Convert HUBO to QUBO for Qiskit
qubo_problem = convert_hubo_to_qubo(rand_hubo_problem, num_variables)



Generated HUBO Problem:
Term: (0, 1, 2), Coefficient: -1.10
Term: (0, 1), Coefficient: -0.64
Term: (1,), Coefficient: 1.07


In [112]:
type(qubo_problem)

qiskit_optimization.problems.quadratic_program.QuadraticProgram

In [114]:
qubo

BinaryQuadraticModel({0: 0.0, 1: 1.0699470414898489, 2: 0.0, 3: 1000.0, 4: 0.0}, {(1, 0): -3.9367225901667897, (2, 0): -3.2954216470412057, (2, 1): -3.2954216470412057, (3, 0): -1000.0, (3, 1): -1000.0, (3, 2): -1000.0, (4, 0): 0.0, (4, 1): 0.0, (4, 2): 0.0, (4, 3): 0.0}, 0.0, 'BINARY')

In [113]:
from qiskit_optimization.algorithms import GurobiOptimizer

start_classical = time.time()
# Use the GurobiOptimizer to find the solution
exact_solver = GurobiOptimizer()
result = exact_solver.solve(qubo_problem)

end_classical = time.time()
# Display the result

print("Optimal solution(s):")
solutions = [s for s in result.samples if s.fval == result.fval]
for solution in solutions:
    print(solution)
    
print("Optimal value:", result.fval)
print("Calculated in:", end_classical - start_classical, "seconds.")

IndexError: tuple index out of range